### There are three use cases that demostrate the usage of PAVLIB4SWAT to render static data, animate tempospatial data, and run and compare different scenarios

###### ------------------------------------------------------------------------------------------

#### Case 1: visualizing static data

##### Step 1: convert shapefiles to geojson

In [1]:
import warnings
import XMUT
warnings.filterwarnings("ignore")
sfp=XMUT.DataProvider.ShapeProvider()
filelist=["rivers","outlets","raingauges","subbasins","tempgauges"]
for filename in filelist:
    sfp.convert(r"./data/shpfile/"+filename+r".shp",r"./data/kepler_data/"+filename+".json",outputproject="epsg:4326")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


##### Step 2 and 3: create map configuration and render static data

In [3]:
import json
import XMUT

longtitude=118.01177670289091
latitude=25.440411737132468
zoom=10.76383577825254
configuration=XMUT.MapConfiguration.Configuration()
sfp=XMUT.DataProvider.ShapeProvider()
center=sfp.getCenter(r"./data/shpfile/subbasins.shp","","epsg:4326")
center.append(zoom)
config=configuration.getBaseConfig(*center)
keplerMap=XMUT.MapGenerator.KeplerMap();

filelist=["rivers","outlets","raingauges","subbasins","tempgauges"]
for filename in filelist:   
    f=open(r"./data/kepler_data/"+filename+".json","r")
    data=json.load(f)
    f.close()
    keplerMap.setConfigure(config=config)
    keplerMap.addLayer(data,filename)
    
keplerMap.map.save_to_html(file_name=r"./map1.html")

Map saved to ./map1.html!


##### Result: If everything works fine, the generated map should be similar to the following graphic
<img src="./data/screen_snapshot/map1.jpg" alt="map1" width="400"/>

###### ------------------------------------------------------------------------------------------

#### Case 2: animate tempospatial data

##### Step 1: The following codes convert output.rch to geojson file

In [5]:
import XMUT
import warnings
warnings.filterwarnings("ignore")
rp=XMUT.DataProvider.RCHProvider()
rp.convert(r"./data/model/output.rch",r"./data/kepler_data/ts_rivers.json",subnum=5,rivergeojson=r"./data/kepler_data/rivers.json",ciofile=r"./data/model/file.cio")

##### Step 2: Generate map configuration for visualizing discharge time series

In [6]:
import XMUT
import warnings
warnings.filterwarnings("ignore")
ciofile=r"./data/model/file.cio"
swatUtil=XMUT.DataProvider.Util.SWATUtil()
startDate=swatUtil.getStartDate(swatiofile=ciofile)
configuration = XMUT.MapConfiguration.Configuration()
config=configuration.getConfigByType("ts_rivers","ts_rivers_id",startDate)

##### Step 3: animate tempospatial data

In [7]:
import XMUT
import json
import warnings
warnings.filterwarnings("ignore")
keplerMap=XMUT.MapGenerator.KeplerMap()
f=open(r"./data/kepler_data/ts_rivers.json","r")
data=json.load(f)
f.close()
keplerMap.addLayer(data,"ts_rivers_id")
keplerMap.setConfigure(config=config) 
  
keplerMap.map.save_to_html(file_name=r"./map2.html")

Map saved to ./map2.html!


##### Result: If everything works fine, the generated map should be similar to the following graphic
<img src="./data/screen_snapshot/map2.jpg" alt="map2" width="400"/>

###### ------------------------------------------------------------------------------------------

#### Case 3: run and compare different scenarios

##### Step 1: run scenario 1. In this scenario the parameter CN2 and ALPHA_BF are adjusted and saved to model inputs, and then model is executed and model output is stored to a file.

In [1]:
import XMUT

paramerterEdit=XMUT.ModelEditingService.ModelParameterEditing()
paramerterEdit.addCalibrationParameter("r__CN2.mgt", "0.8")
paramerterEdit.addCalibrationParameter("v__ALPHA_BF.gw", "0.94")
paramerterEdit.saveToModelIn("./data/model/")
paramerterEdit.invokeModel("./data/model/","./data/model/output.bak1.rch")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Command executed successfully
Output: None
Command executed successfully
Output:                 SWAT2009               
               Rev. 481               
      Soil & Water Assessment Tool    
               PC Version             
 Program reading from file.cio . . . executing

  Executing year    1
  Executing year    2
  Executing year    3
  Executing year    4
  Executing year    5
  Executing year    6
  Executing year    7
  Executing year    8

 Execution successfully completed 

Backup created successfully: ./data/model/output.bak1.rch


##### Step 2: run scenario 2. Similar procedures of Step 1 are performed except the parameter values are different to those of the step 1.

In [2]:
import XMUT

paramerterEdit=XMUT.ModelEditingService.ModelParameterEditing()
paramerterEdit.addCalibrationParameter("r__CN2.mgt", "1.2")
paramerterEdit.addCalibrationParameter("v__ALPHA_BF.gw", "0.54")
paramerterEdit.saveToModelIn("./data/model/")
paramerterEdit.invokeModel("./data/model/","./data/model/output.bak2.rch")


Command executed successfully
Output: None
Command executed successfully
Output:                 SWAT2009               
               Rev. 481               
      Soil & Water Assessment Tool    
               PC Version             
 Program reading from file.cio . . . executing

  Executing year    1
  Executing year    2
  Executing year    3
  Executing year    4
  Executing year    5
  Executing year    6
  Executing year    7
  Executing year    8

 Execution successfully completed 

Backup created successfully: ./data/model/output.bak2.rch


##### Step 3: two scenarios' results are merged and the merged data is associated with river outlets.

In [3]:
%matplotlib inline
import XMUT
import warnings
warnings.filterwarnings("ignore")
rp=XMUT.DataProvider.RCHScenarioProvider()
rp.convert(r"./data/model/output.bak1.rch",r"./data/kepler_data/ts_scenarios.json",subnum=5,outlets=r"./data/kepler_data/outlets.json",ciofile=r"./data/model/file.cio",inputfile2=r"./data/model/output.bak2.rch")

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

##### Step 4: generate map configuration.

In [4]:
import XMUT
import warnings
warnings.filterwarnings("ignore")
ciofile=r"./data/model/file.cio"
swatUtil=XMUT.DataProvider.Util.SWATUtil()
startDate=swatUtil.getStartDate(swatiofile=ciofile)
configuration = XMUT.MapConfiguration.Configuration()
config1=configuration.getConfigByType("rivers","rivers_id",startDate)
config2=configuration.getConfigByType("scenarios","ts_scenarios_id",startDate)
config=configuration.mergeConfig(config1,config2)

##### Step 5: visualizing scenario results in map.

In [5]:
import XMUT
import json
import warnings
warnings.filterwarnings("ignore")
keplerMap=XMUT.MapGenerator.KeplerMap()
f=open(r"./data/kepler_data/ts_scenarios.json","r")
data=json.load(f)
f.close()
keplerMap.addLayer(data,"ts_scenarios_id")

f=open(r"./data/kepler_data/rivers.json","r")
data=json.load(f)
f.close()
keplerMap.addLayer(data,"rivers_id")

keplerMap.setConfigure(config=config) 
keplerMap.map.save_to_html(file_name=r"./map3.html")

Map saved to ./map3.html!


##### Result: If everything works fine, the generated map should be similar to the following graphic
<img src="./data/screen_snapshot/map3.jpg" alt="map3" width="400"/>